*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# The Microsoft Toolkit of Multi-Task Deep Neural Networks for Natural Language Understanding


## Summary

MT-DNN is an open-source natural language understanding (NLU) toolkit that makes it easy for researchers and developers to train customized deep learning models. Built upon PyTorch and Transformers, MT-DNN is designed to facilitate rapid
customization for a broad spectrum of NLU tasks, using a variety of objectives (classification, regression, structured prediction) and text encoders (e.g., RNNs, BERT, RoBERTa, UniLM). A unique feature of MT-DNN is its built-in support for robust and transferable learning using the adversarial multi-task learning paradigm. To enable efficient production deployment, MT-DNN supports multitask knowledge distillation, which can substantially compress a deep neural model without significant performance drop. We demonstrate the effectiveness of MT-DNN on a wide range of NLU applications across general and biomedical domains. The pip installable package and pretrained models will be publicly available at https://github.com/microsoft/mt-dnn.

### Design

MT-DNN is designed for modularity, flexibility, and ease of use. These modules are built upon PyTorch (Paszke et al., 2019) and Transformers (Wolf
et al., 2019), allowing the use of the SOTA pretrained models, e.g., BERT (Devlin et al., 2019), RoBERTa (Liu et al., 2019c) and UniLM (Dong
et al., 2019). The unique attribute of this package is a flexible interface for adversarial multi-task fine-tuning and knowledge distillation, so that researchers and developers can build large SOTA NLU models and then compress them to small ones
for online deployment.The overall workflow and system architecture are shown in figures 1 and 3 respectively.


![Workflow Design](https://nlpbp.blob.core.windows.net/images/mt-dnn2.JPG)

The above figure shows workflow of MT-DNN: train a neural language model on a large amount of unlabeled raw text
to obtain general contextual representations; then finetune the learned contextual representation on downstream tasks, e.g. GLUE (Wang et al., 2018); lastly, distill this large model to a lighter one for online deployment. In the later two phrases, we can leverage powerful multi-task learning and adversarial training to further improve performance.

## Architecture

![overall_arch](https://nlpbp.blob.core.windows.net/images/mt-dnn.png)
The figure above shows the overall system architecture. The lower layers are shared across all tasks while the top layers are taskspecific. The input X (either a sentence or a set of sentences) is first represented as a sequence of embedding
vectors, one for each word, in l1. Then the encoder, e.g a Transformer or recurrent neural network (LSTM) model,
captures the contextual information for each word and generates the shared contextual embedding vectors in l2.
Finally, for each task, additional task-specific layers generate task-specific representations, followed by operations
necessary for classification, similarity scoring, or relevance ranking. In case of adversarial training, we perturb
embeddings from the lexicon encoder and then add an extra loss term during the training. Note that for the
inference phrase, it does not require perturbations.

## Introduction
In this notebook, we fine-tune and evaluate MT-DNN models on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.  

### Running Time

This is a __computationally intensive__ notebook that runs on the entire MNLI dataset for match and mismatched datasets for training, development and test.  

The table below provides some reference running time on a GPU machine.  

|Dataset|MULTI_GPU_ON|Machine Configurations|Running time|
|:------|:---------|:----------------------|:------------|
|MultiNLI|True|4 NVIDIA Tesla K80 GPUs, 24GB GPU memory| ~ 20 hours |

If you run into `CUDA out-of-memory error` or the jupyter kernel dies constantly, try reducing the `BATCH_SIZE` and `MAX_SEQ_LEN` in `MTDNNConfig`, but note that model performance may be compromised.



### Text Classification of MultiNLI Sentences using MT-DNN

This notebook utilizes the pip installable package that implements the Multi-Task Deep Neural Network Toolkit (MTDNN) for Natural Language Understanding. It's recommended to run this notebook on GPU machines as it's very computationally intensive.

In [ ]:
!git clone https://www.github.com/nvidia/apex
cd apex
!python3 setup.py install

In [ ]:
!git clone https://github.com/microsoft/MT-DNN.git

Cloning into 'MT-DNN'...
remote: Enumerating objects: 524, done.
remote: Total 524 (delta 0), reused 0 (delta 0), pack-reused 524
Receiving objects: 100% (524/524), 185.43 KiB | 4.75 MiB/s, done.
Resolving deltas: 100% (297/297), done.


In [ ]:
cd /content/MT-DNN

/content/MT-DNN


In [ ]:
!pip install -e /content/MT-DNN

Obtaining file:///content/MT-DNN
  Found existing installation: mtdnn 1.1.0
    Can't uninstall 'mtdnn'. No files were found to uninstall.
  Running setup.py develop for mtdnn


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import shutil
import sys
from tempfile import TemporaryDirectory

import pandas as pd
import torch

from mtdnn.common.types import EncoderModelType
from mtdnn.configuration_mtdnn import MTDNNConfig
from mtdnn.data_builder_mtdnn import MTDNNDataBuilder
from mtdnn.modeling_mtdnn import MTDNNModel
from mtdnn.process_mtdnn import MTDNNDataProcess
from mtdnn.tasks.config import MTDNNTaskDefs
from mtdnn.tokenizer_mtdnn import MTDNNTokenizer

## Define Configuration, Tasks and Model Objects

In [ ]:
# Define Configuration, Tasks and Model Objects
ROOT_DIR = TemporaryDirectory().name
OUTPUT_DIR = os.path.join(ROOT_DIR, 'checkpoint')
os.makedirs(OUTPUT_DIR) if not os.path.exists(OUTPUT_DIR) else OUTPUT_DIR

LOG_DIR = os.path.join(ROOT_DIR, 'tensorboard_logdir')
os.makedirs(LOG_DIR) if not os.path.exists(LOG_DIR) else LOG_DIR

DATA_DIR = "../../sample_data/"
DATA_SOURCE_DIR = os.path.join(DATA_DIR, "MNLI")

# Training parameters
BATCH_SIZE = 16
MULTI_GPU_ON = True
MAX_SEQ_LEN = 128
NUM_EPOCHS = 5

Exploring the location for our data to be downloaded, model to be checkpointed and logs to be dumped

In [ ]:
print(OUTPUT_DIR)
print(LOG_DIR)

/tmp/tmphpd9y4rp/checkpoint
/tmp/tmphpd9y4rp/tensorboard_logdir


## Read Dataset
We start by loading a subset of the data. The MNLI dataset has been downloaded for your convenience and added as part of this repository. The data can be found in `../../sample_data/MNLI`.   

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators.  

> **NOTE:** The MNLI data is very large and would need [Git LFS](https://docs.github.com/en/github/managing-large-files/installing-git-large-file-storage) installed on your machine to pull it down.  


In [ ]:
assert os.path.exists(DATA_SOURCE_DIR), "[ERROR] - The MNLI Dataset cannot be found."
os.listdir(DATA_SOURCE_DIR)

AssertionError: ignored

### Define a Configuration Object 

Create a model configuration object, `MTDNNConfig`, with the necessary parameters to initialize the MT-DNN model. Initialization without any parameters will default to a similar configuration that initializes a BERT model. 

In [ ]:
config = MTDNNConfig(batch_size=BATCH_SIZE, 
                     max_seq_len=MAX_SEQ_LEN, 
                     multi_gpu_on=MULTI_GPU_ON)


### Create Task Definition Object  

Define the task parameters to train for and initialize an `MTDNNTaskDefs` object. Create a task parameter dictionary. Definition can be a single or multiple tasks to train.  `MTDNNTaskDefs` can take a python dict, yaml or json file with task(s) defintion.

The data source directory is the path of data downloaded and extracted above using `download_tsv_files_and_extract` which is the `MNLI` dir under the `DATA_DIR` temporary directory.    

The data source has options that are set to drive each task pre-processing; `data_process_opts`


In [ ]:
tasks_params = {
    "mnli": {
        "data_format": "PremiseAndOneHypothesis",
        "encoder_type": "BERT",
        "dropout_p": 0.3,
        "enable_san": True,
        "labels": ["contradiction", "neutral", "entailment"],
        "metric_meta": ["ACC"],
        "loss": "CeCriterion",
        "kd_loss": "MseCriterion",
        "n_class": 3,
        "split_names": [
            "train",
            "dev_matched",
            "dev_mismatched",
            "test_matched",
            "test_mismatched",
        ],
        "data_source_dir": DATA_SOURCE_DIR,
        "data_process_opts": {"header": True, "is_train": True, "multi_snli": False,},
        "task_type": "Classification",
    },
}

# Define the tasks
task_defs = MTDNNTaskDefs(tasks_params)

07/01/2020 04:42:22 - mtdnn.tasks.config - INFO - Mapping Task attributes
07/01/2020 04:42:22 - mtdnn.tasks.config - INFO - Configured task definitions - ['mnli']



### Create the MTDNN Data Tokenizer Object  

Create a data tokenizing object, `MTDNNTokenizer`. Based on the model initial checkpoint, it wraps around the model's Huggingface transformers library to encode the data to MT-DNN format. This becomes the input to the data building stage.  


In [ ]:
tokenizer = MTDNNTokenizer(do_lower_case=True)

#### Testing out the Tokenizer encode function on a sample text
`tokenizer.encode("What NLP toolkit do you recommend", "MT-DNN is a fantastic toolkit")`

In [ ]:
print(tokenizer.encode("What NLP toolkit do you recommend", "MT-DNN is a fantastic toolkit"))

([101, 2054, 17953, 2361, 6994, 23615, 2079, 2017, 16755, 102, 11047, 1011, 1040, 10695, 2003, 1037, 10392, 6994, 23615, 102], None, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


## Data Preprocessing

### Create the Data Builder Object  

Create a data preprocessing object, `MTDNNDataBuilder`. This class is responsible for converting the data into the MT-DNN format depending on the task.  
 

Define a data builder that handles the creating of each task's vectorized data utilizing the model tokenizer. This will build out the vectorized data needed for creating the training, test and development PyTorch dataloaders

In [ ]:
## Load and build data
data_builder = MTDNNDataBuilder(
    tokenizer=tokenizer,
    task_defs=task_defs,
    data_dir=DATA_SOURCE_DIR,
    canonical_data_suffix="canonical_data",
    dump_rows=True,
)

## Build data to MTDNN Format
## Iterable of each specific task and processed data
vectorized_data = data_builder.vectorize()

06/30/2020 07:00:07 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 392702 samples for mnli at ../../sample_data/MNLI/canonical_data/mnli_train.tsv
06/30/2020 07:00:08 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9815 samples for mnli at ../../sample_data/MNLI/canonical_data/mnli_dev_matched.tsv
06/30/2020 07:00:08 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9832 samples for mnli at ../../sample_data/MNLI/canonical_data/mnli_dev_mismatched.tsv
06/30/2020 07:00:08 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9796 samples for mnli at ../../sample_data/MNLI/canonical_data/mnli_test_matched.tsv
06/30/2020 07:00:08 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9847 samples for mnli at ../../sample_data/MNLI/canonical_data/mnli_test_mismatched.tsv
mnli_train
06/30/2020 07:00:08 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI TRAIN' Task


Building Data For Premise and One Hypothesis: 392702it [05:08, 1272.59it/s]

06/30/2020 07:05:18 - mtdnn.data_builder_mtdnn - INFO - Saving data to ../../sample_data/MNLI/canonical_data/bert_base_uncased/mnli_train.json



Saving Data For PremiseAndOneHypothesis: 100%|██████████| 392702/392702 [00:05<00:00, 68365.81it/s]

mnli_dev_matched
06/30/2020 07:05:24 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI DEV MATCHED' Task


...


Saving Data For PremiseAndOneHypothesis: 100%|██████████| 9796/9796 [00:00<00:00, 71259.18it/s]

mnli_test_mismatched
06/30/2020 07:05:47 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI TEST MISMATCHED' Task



Building Data For Premise and One Hypothesis: 9847it [00:07, 1242.78it/s]

06/30/2020 07:05:55 - mtdnn.data_builder_mtdnn - INFO - Saving data to ../../sample_data/MNLI/canonical_data/bert_base_uncased/mnli_test_mismatched.json



Saving Data For PremiseAndOneHypothesis: 100%|██████████| 9847/9847 [00:00<00:00, 69755.60it/s]


### Create the Data Processing Object  

Create a data preprocessing object, `MTDNNDataProcess`. This creates the training, test and development PyTorch dataloaders needed for training and testing. We also need to retrieve the necessary training options required to initialize the model correctly, for all tasks.  

Define a data process that handles creating the training, test and development PyTorch dataloaders

In [ ]:
# Make the Data Preprocess step and update the config with training data updates
data_processor = MTDNNDataProcess(
    config=config, task_defs=task_defs, vectorized_data=vectorized_data
)

07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Starting to process the training data sets
07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Loading mnli_train as task 0
07/01/2020 04:42:36 - mtdnn.dataset_mtdnn - INFO - Loaded 391533 samples out of 392702
07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Starting to process the testing data sets
07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Loading mnli_dev_matched as task 0
07/01/2020 04:42:36 - mtdnn.dataset_mtdnn - INFO - Loaded 9815 samples out of 9815
07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Loading mnli_dev_mismatched as task 0
07/01/2020 04:42:36 - mtdnn.dataset_mtdnn - INFO - Loaded 9832 samples out of 9832
07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Loading mnli_test_matched as task 0
07/01/2020 04:42:36 - mtdnn.dataset_mtdnn - INFO - Loaded 9796 samples out of 9796
07/01/2020 04:42:36 - mtdnn.process_mtdnn - INFO - Loading mnli_test_mismatched as task 0
07/01/2020 04:42:36 - mtdnn.dataset_mtdnn - I

Retrieve the processed batch multitask batch data loaders for training, development and test

In [ ]:
multitask_train_dataloader = data_processor.get_train_dataloader()
dev_dataloaders_list = data_processor.get_dev_dataloaders()
test_dataloaders_list = data_processor.get_test_dataloaders()

Now we can retrieve the training options, from the processor, to initialize model with.

In [ ]:
decoder_opts = data_processor.get_decoder_options_list()
task_types = data_processor.get_task_types_list()
dropout_list = data_processor.get_tasks_dropout_prob_list()
loss_types = data_processor.get_loss_types_list()
kd_loss_types = data_processor.get_kd_loss_types_list()
tasks_nclass_list = data_processor.get_task_nclass_list()

Let us update the batch steps

In [ ]:
num_all_batches = data_processor.get_num_all_batches()

### Instantiate the MTDNN Model

Now we can go ahead and create an `MTDNNModel` model

In [ ]:
model = MTDNNModel(
    config,
    task_defs,
    pretrained_model_name="bert-base-uncased",
    num_train_step=num_all_batches,
    decoder_opts=decoder_opts,
    task_types=task_types,
    dropout_list=dropout_list,
    loss_types=loss_types,
    kd_loss_types=kd_loss_types,
    tasks_nclass_list=tasks_nclass_list,
    multitask_train_dataloader=multitask_train_dataloader,
    dev_dataloaders_list=dev_dataloaders_list,
    test_dataloaders_list=test_dataloaders_list,
    output_dir=OUTPUT_DIR,
    log_dir=LOG_DIR 
)

idx: 0, number of task labels: 3


## Model Finetuning, Prediction and Evaluation

### Fit and finetune model on five epochs and predict using the training and test  

At this point the MT-DNN model allows us to fit to the model and create predictions. The fit takes an optional `epochs` parameter that overwrites the epochs set in the `MTDNNConfig` object. 

In [ ]:
model.fit(epochs=NUM_EPOCHS)

06/26/2020 08:14:07 - mtdnn.modeling_mtdnn - INFO - Total number of params: 109484547
06/26/2020 08:14:07 - mtdnn.modeling_mtdnn - INFO - At epoch 0
06/26/2020 08:14:07 - mtdnn.modeling_mtdnn - INFO - Amount of data to go over: 24471
06/26/2020 08:14:13 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [     1] Training Loss - [1.63923] Time Remaining - [1 day, 13:33:33]
06/26/2020 08:19:40 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [   500] Training Loss - [1.32204] Time Remaining - [4:25:55]
06/26/2020 08:25:09 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  1000] Training Loss - [1.21343] Time Remaining - [4:18:55]
06/26/2020 08:30:42 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  1500] Training Loss - [1.16369] Time Remaining - [4:13:52]
06/26/2020 08:36:15 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  2000] Training Loss - [1.12522] Time Remaining - [4:08:36]
06/26/2020 08:41:48 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  2

06/27/2020 01:25:00 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 28000] Training Loss - [0.59204] Time Remaining - [3:52:28]
06/27/2020 01:30:34 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 28500] Training Loss - [0.58952] Time Remaining - [3:46:57]
06/27/2020 01:36:07 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 29000] Training Loss - [0.58707] Time Remaining - [3:41:27]
06/27/2020 01:41:39 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 29500] Training Loss - [0.58480] Time Remaining - [3:35:47]
06/27/2020 01:47:11 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 30000] Training Loss - [0.58238] Time Remaining - [3:30:10]
06/27/2020 01:52:43 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 30500] Training Loss - [0.57984] Time Remaining - [3:24:34]
06/27/2020 01:58:16 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [ 31000] Training Loss - [0.57737] Time Remaining - [3:19:04]
06/27/2020 02:03:47 - mtdnn.modeling_mtdnn - INF

### Evaluation and Prediction
Perform inference using the last (best) checkpointed model. With 5 epochs, the last model would be `model_4.pt`

In [ ]:
model.predict(trained_model_chckpt=f"{OUTPUT_DIR}/model_4.pt")

06/27/2020 06:51:24 - mtdnn.modeling_mtdnn - INFO - Running predictions using: /tmp/tmpd9ok4aeo/checkpoint/model_4.pt
06/27/2020 06:51:25 - mtdnn.modeling_mtdnn - INFO - predicting 0
06/27/2020 06:51:45 - mtdnn.modeling_mtdnn - INFO - predicting 100
06/27/2020 06:52:05 - mtdnn.modeling_mtdnn - INFO - predicting 200
06/27/2020 06:52:27 - mtdnn.modeling_mtdnn - INFO - predicting 300
06/27/2020 06:52:47 - mtdnn.modeling_mtdnn - INFO - predicting 400
06/27/2020 06:53:07 - mtdnn.modeling_mtdnn - INFO - predicting 500
06/27/2020 06:53:28 - mtdnn.modeling_mtdnn - INFO - predicting 600
06/27/2020 06:53:48 - mtdnn.modeling_mtdnn - INFO - predicting 700
06/27/2020 06:54:10 - mtdnn.modeling_mtdnn - INFO - predicting 800
06/27/2020 06:54:30 - mtdnn.modeling_mtdnn - INFO - predicting 900
06/27/2020 06:54:50 - mtdnn.modeling_mtdnn - INFO - predicting 1000
06/27/2020 06:55:11 - mtdnn.modeling_mtdnn - INFO - predicting 1100
06/27/2020 06:55:31 - mtdnn.modeling_mtdnn - INFO - predicting 1200
06/27/2020

### Results

In [ ]:
results = {}
dev_result_files = list(filter(lambda x: x.endswith('.json') and 'dev' in x, os.listdir(OUTPUT_DIR))) 
for d in dev_result_files: 
    name =  ' '.join(list(map(str.capitalize, d.split('_')))[:3]) 
    file_name = os.path.join(OUTPUT_DIR, d)
    with open(file_name, 'r') as f: 
        res = json.load(f) 
        results.update(
            {name: {
                'ACCURACY': f"{res['metrics']['ACC']:.3f}"
                }
            }) 
df_results = pd.DataFrame(results)   
df_results

,Mnli Mismatched Dev,Mnli Matched Dev
ACCURACY,84.422,84.144


## Clean up temporary folders

In [ ]:
if os.path.exists(ROOT_DIR):
    shutil.rmtree(ROOT_DIR, ignore_errors=True)